In [2]:
import pandas as pd

# Read the merged_player_data.csv file
data = pd.read_csv('merged_player_data.csv')

# Calculate the Points_Percent_Diff column and round to five decimal places
data['Points_Percent_Diff'] = round(data['Points_Diff'] / data['Points_Running_Avg'], 5)

# Calculate the Rebounds_Percent_Diff column and round to five decimal places
data['Rebounds_Percent_Diff'] = round(data['Rebounds_Diff'] / data['Rebounds_Running_Avg'], 5)

# Calculate the Assists_Percent_Diff column and round to five decimal places
data['Assists_Percent_Diff'] = round(data['Assists_Diff'] / data['Assists_Running_Avg'], 5)

# Write the updated dataframe to percent_diff.csv
data.to_csv('percent_diff.csv', index=False)


In [13]:
# Read the merged_player_data.csv file
data = pd.read_csv('merged_player_data.csv')

# Group the data by Player column
grouped_data = data.groupby('Player')

# Calculate the standard deviation of Points_Running_Avg, Rebounds_Running_Avg, and Assists_Running_Avg
std_points = round(grouped_data['PTS'].std(), 5)
std_rebounds = round(grouped_data['TRB'].std(), 5)
std_assists = round(grouped_data['AST'].std(), 5)

# Create new columns for the standard deviation of each
data['Points_Running_Avg_Std'] = data['Player'].map(std_points)
data['Rebounds_Running_Avg_Std'] = data['Player'].map(std_rebounds)
data['Assists_Running_Avg_Std'] = data['Player'].map(std_assists)

# Write the updated dataframe to percent_diff.csv
data.to_csv('stdev.csv', index=False)



In [14]:

# Read the stdev.csv file
data = pd.read_csv('stdev.csv')

# Calculate the z-score for PTS
data['PTS_z'] = round((data['PTS'] - data['Points_Running_Avg']) / data['Points_Running_Avg_Std'], 5)

# Calculate the z-score for TRB
data['TRB_z'] = round((data['TRB'] - data['Rebounds_Running_Avg']) / data['Rebounds_Running_Avg_Std'], 5)

# Calculate the z-score for AST
data['AST_z'] = round((data['AST'] - data['Assists_Running_Avg']) / data['Assists_Running_Avg_Std'], 5)

# Write the updated dataframe to z_score.csv
data.to_csv('z_score.csv', index=False)


In [15]:
import numpy as np

In [16]:
# Read the z_score.csv file
data = pd.read_csv('z_score.csv')

# Add columns for bet results
data['PTS_Bet_Res'] = np.where(data['PTS'] > data['player_points_line'], 'Over', 'Under')
data['TRB_Bet_Res'] = np.where(data['TRB'] > data['player_rebounds_line'], 'Over', 'Under')
data['AST_Bet_Res'] = np.where(data['AST'] > data['player_assists_line'], 'Over', 'Under')

# Write the updated dataframe to bet_result.csv
data.to_csv('bet_result.csv', index=False)



In [3]:
import pandas as pd
import numpy as np

In [4]:


# Read the bet_result.csv file
data = pd.read_csv('bet_result.csv')


# Loop through the range of z-scores from 0.5 to 2.0 in increments of 0.1
for z_thresh in np.arange(0.5, 2.0, 0.1):
    # Add the prev_performance column based on the conditions and values
    data['pred_prev'] = np.where(data['PTS_z'] >= z_thresh, "Over", "NA")

    # Shift the values of pred_prev down by 1
    data['pred_prev'] = data['pred_prev'].shift(1)

    # Find the row with the smallest G for each player
    min_G_rows = data.groupby('Player')['G'].idxmin()

    # Replace the prev_pred in the rows with the smallest G with "NA"
    data.loc[min_G_rows, 'pred_prev'] = "NA"


    # Count the number of rows where pred_prev is not "NA" and is equal to PTS_Bet_Res
    success_count = len(data[(data['pred_prev'] == 'Over') & (data['PTS_Bet_Res'] == 'Over')])

    count_pred_prev_not_na = len(data[data['pred_prev'] == 'Over'])

    # Calculate the success rate
    success_rate = round(success_count / count_pred_prev_not_na, 5)

    print(f"{round(z_thresh, 1)}    {success_rate}    {success_count}")



# Write the updated dataframe to bet_result.csv
#data.to_csv('prev_perf.csv', index=False)


0.5    0.53789    291
0.6    0.53388    260
0.7    0.49386    201
0.8    0.5    186
0.9    0.51515    170
1.0    0.53737    151
1.1    0.53906    138
1.2    0.56054    125
1.3    0.57143    116
1.4    0.58621    102
1.5    0.5906    88
1.6    0.5873    74
1.7    0.58879    63
1.8    0.59036    49
1.9    0.6087    42


In [12]:
# Read the bet_result.csv file
data = pd.read_csv('bet_result.csv')


# Loop through the range of z-scores from 0.5 to 2.0 in increments of 0.1
for z_thresh in np.arange(-2.0, 2.0, 0.1):
    # Add the prev_performance column based on the conditions and values
    data = data[data['Points_Running_Avg'] > 25]

    data['pred_prev'] = np.where(data['PTS_z'] <= -z_thresh, "Under", "NA")

    # Shift the values of pred_prev down by 1
    data['pred_prev'] = data['pred_prev'].shift(1)

    # Find the row with the smallest G for each player
    min_G_rows = data.groupby('Player')['G'].idxmin()

    # Replace the prev_pred in the rows with the smallest G with "NA"
    data.loc[min_G_rows, 'pred_prev'] = "NA"


    # Count the number of rows where pred_prev is not "NA" and is equal to PTS_Bet_Res
    success_count = len(data[(data['pred_prev'] == 'Under') & (data['PTS_Bet_Res'] == 'Under')])

    count_pred_prev_not_na = len(data[data['pred_prev'] == 'Under'])

    # Calculate the success rate
    success_rate = round(success_count / count_pred_prev_not_na, 5)

    print(f"{round(-z_thresh, 1)}    {success_rate}    {success_count}")

2.0    0.55738    136
1.9    0.55187    133
1.8    0.55462    132
1.7    0.55274    131
1.6    0.55508    131
1.5    0.55603    129
1.4    0.55947    127
1.3    0.56195    127
1.2    0.56502    126
1.1    0.56164    123
1.0    0.56279    121
0.9    0.55769    116
0.8    0.5495    111
0.7    0.53807    106
0.6    0.54787    103
0.5    0.54237    96
0.4    0.56364    93
0.3    0.56209    86
0.2    0.55556    80
0.1    0.57252    75
-0.0    0.55285    68
-0.1    0.55856    62
-0.2    0.56436    57
-0.3    0.57955    51
-0.4    0.58442    45
-0.5    0.59722    43
-0.6    0.59091    39
-0.7    0.62712    37
-0.8    0.60784    31
-0.9    0.63636    28
-1.0    0.62162    23
-1.1    0.62857    22
-1.2    0.64286    18
-1.3    0.64    16
-1.4    0.6087    14
-1.5    0.57895    11
-1.6    0.5625    9
-1.7    0.54545    6
-1.8    0.55556    5
-1.9    0.4    2


In [10]:


# Read the bet_result.csv file
data = pd.read_csv('bet_result.csv')


# Loop through the range of z-scores from 0.5 to 2.0 in increments of 0.1
for z_thresh in np.arange(0.5, 2.0, 0.1):
    # Remove all rows from data where PTS column is less than 20
    data = data[data['Points_Running_Avg'] >= 25]

    # Add the prev_performance column based on the conditions and value
    data['pred_prev'] = np.where(data['PTS_z'] >= z_thresh, "Over", "NA")

    # Shift the values of pred_prev down by 1
    data['pred_prev'] = data['pred_prev'].shift(1)

    # Find the row with the smallest G for each player
    min_G_rows = data.groupby('Player')['G'].idxmin()

    # Replace the prev_pred in the rows with the smallest G with "NA"
    data.loc[min_G_rows, 'pred_prev'] = "NA"


    # Count the number of rows where pred_prev is not "NA" and is equal to PTS_Bet_Res
    success_count = len(data[(data['pred_prev'] == 'Over') & (data['PTS_Bet_Res'] == 'Over')])

    count_pred_prev_not_na = len(data[data['pred_prev'] == 'Over'])

    # Calculate the success rate
    success_rate = round(success_count / count_pred_prev_not_na, 5)

    print(f"{round(z_thresh, 1)}    {success_rate}    {success_count}")

0.5    0.4    30
0.6    0.40625    26
0.7    0.36364    20
0.8    0.4    20
0.9    0.43182    19
1.0    0.45946    17
1.1    0.45455    15
1.2    0.48276    14
1.3    0.46154    12
1.4    0.45833    11
1.5    0.42105    8
1.6    0.4    6
1.7    0.35714    5
1.8    0.38462    5
1.9    0.3    3


In [11]:

# Read the bet_result.csv file
data = pd.read_csv('bet_result.csv')


# Loop through the range of z-scores from 0.5 to 2.0 in increments of 0.1
for z_thresh in np.arange(0.5, 2.0, 0.1):
    # Remove all rows from data where PTS column is less than 20
    data = data[data['Points_Running_Avg'] < 25]

    # Add the prev_performance column based on the conditions and value
    data['pred_prev'] = np.where(data['PTS_z'] >= z_thresh, "Over", "NA")

    # Shift the values of pred_prev down by 1
    data['pred_prev'] = data['pred_prev'].shift(1)

    # Find the row with the smallest G for each player
    min_G_rows = data.groupby('Player')['G'].idxmin()

    # Replace the prev_pred in the rows with the smallest G with "NA"
    data.loc[min_G_rows, 'pred_prev'] = "NA"


    # Count the number of rows where pred_prev is not "NA" and is equal to PTS_Bet_Res
    success_count = len(data[(data['pred_prev'] == 'Over') & (data['PTS_Bet_Res'] == 'Over')])

    count_pred_prev_not_na = len(data[data['pred_prev'] == 'Over'])

    # Calculate the success rate
    success_rate = round(success_count / count_pred_prev_not_na, 5)

    print(f"{round(z_thresh, 1)}    {success_rate}    {success_count}")

0.5    0.55724    258
0.6    0.54869    231
0.7    0.51143    179
0.8    0.5125    164
0.9    0.52817    150
1.0    0.54959    133
1.1    0.55204    122
1.2    0.56995    110
1.3    0.58523    103
1.4    0.60403    90
1.5    0.6124    79
1.6    0.60909    67
1.7    0.61957    57
1.8    0.62319    43
1.9    0.65517    38


In [20]:

# Read the prev_perf.csv file
data = pd.read_csv('prev_perf.csv')

# Count the number of rows where pred_prev is not "NA" and is equal to PTS_Bet_Res
success_count = len(data[(data['pred_prev'] != 'NA') & (data['pred_prev'] == data['PTS_Bet_Res'])])

count_pred_prev_not_na = len(data[data['pred_prev'] == 'Under'])

# Calculate the success rate
success_rate = round(success_count / count_pred_prev_not_na, 5)

print(success_rate)


0.46263
